In [1]:
!nvidia-smi

Sat Oct 31 12:49:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !pip install -q torch==1.2.0 torchvision==0.4.0
# !pip install pillow==6.1.0

In [4]:
import torch
import PIL 
print(torch.__version__)
print(PIL.__version__)

1.2.0
6.1.0


In [5]:
%cd /content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch

/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch


In [6]:
# !git clone https://github.com/ekosman/AnomalyDetectionCVPR2018-Pytorch.git

In [7]:
import argparse
import os
from os import path
import torch
import torch.backends.cudnn as cudnn

from utils import metric
from network.anomaly_detector_model import AnomalyDetector, custom_objective, RegularizedLoss
from features_loader import FeaturesLoader
from utils.lr_scheduler import MultiFactorScheduler
from network.model import model
from utils.utils import set_logger

In [10]:
debug_mode = True
features_path = '/content/drive/My Drive/C3D_mini'
annotation_path = "/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/Train_Annotation.txt"
annotation_path_test =  "/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/Test_Annotation.txt"
clip_length = 16
train_frame_interval = 2
val_frame_interval = 2
task_name = 'mini_c3d'
model_dir = '/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/exps/'
log_file = '/content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/log.log'
gpus = "0"
resume_epoch =-1

# optimization
fine_tune = True
batch_size = 60 
lr_base = 0.01
lr_mult_old_layers = 0.2
weight_decay = 0.0001
lr_steps = [int(1e4 * x) for x in [5, 10, 15]]
lr_factor = 1
save_frequency = 10
end_epoch = 20000
random_seed = 1

In [9]:
if __name__ == "__main__":
    set_logger(log_file=log_file, debug_mode=True)

    torch.manual_seed(random_seed)

    train_loader = FeaturesLoader(features_path=features_path,
                                  annotation_path=annotation_path)
    """
    val_loader = FeaturesLoader(features_path=features_path,
                                annotation_path=annotation_path_test)
    """
    train_iter = torch.utils.data.DataLoader(train_loader,
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=8,  # 4, # change this part accordingly
                                             pin_memory=True)
    """
    eval_iter = torch.utils.data.DataLoader(val_loader,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            num_workers=1,  # 4, # change this part accordingly
                                            pin_memory=True)
    """
    iter_seed = torch.initial_seed() + 100

    network = AnomalyDetector()
    net = model(net=network,
                criterion=RegularizedLoss(network, custom_objective).cuda(),
                model_prefix=model_dir,
                step_callback_freq=5,
                save_checkpoint_freq=save_frequency,
                opt_batch_size=batch_size,  # optional, 60 in the paper
                )

    if torch.cuda.is_available():
        net.net.cuda()
        torch.cuda.manual_seed(random_seed)
        net.net = torch.nn.DataParallel(net.net).cuda()
    """
        In the original paper:
        lr = 0.01
        epsilon = 1e-8
    """
    optimizer = torch.optim.Adadelta(net.net.parameters(),
                                     lr=lr_base,
                                     eps=1e-8)

    if resume_epoch < 0:
        epoch_start = 0
        step_counter = 0
    else:
        # net.load_checkpoint(epoch=resume_epoch, optimizer=optimizer)
        epoch_start = resume_epoch
        step_counter = epoch_start * train_iter.__len__()

    # set learning rate scheduler
    num_worker = 1
    lr_scheduler = MultiFactorScheduler(base_lr=lr_base,
                                        steps=[int(x / (batch_size * num_worker)) for x in lr_steps],
                                        factor=lr_factor,
                                        step_counter=step_counter)
    # define evaluation metric
    metrics = metric.MetricList(metric.Loss(name="loss-ce"), )
    # enable cudnn tune
    cudnn.benchmark = True

    net.fit(train_iter=train_iter,
            eval_iter=None,
            optimizer=optimizer,
            lr_scheduler=lr_scheduler,
            metrics=metrics,
            epoch_start=epoch_start,
            epoch_end=end_epoch)

2020-10-31 12:49:37: VideoIter:: iterator initialized (phase: '<NO_NAME>', num: 45)
2020-10-31 12:49:42: Iter 0: start with learning rate: 1.00000e-02 (next lr step: 833)
2020-10-31 12:49:42: Start epoch 0:
2020-10-31 12:50:02: Epoch [0]   Batch [0]    Speed   3.1 (+1479) sample/sec  loss-ce = 1.04472  
2020-10-31 12:50:02: Epoch [0]   time cost: 19.38 sec (0.01 h)
2020-10-31 12:50:02: mkdir /content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/exps
2020-10-31 12:50:02: Checkpoint (model & optimizer) saved to: /content/drive/My Drive/AnomalyDetectionCVPR2018-Pytorch/exps/_ep-0001.pth
2020-10-31 12:50:02: Start epoch 1:
2020-10-31 12:50:08: Epoch [1]   Batch [0]    Speed   9.9 (+1086) sample/sec  loss-ce = 1.04031  
2020-10-31 12:50:08: Epoch [1]   time cost: 6.23 sec (0.00 h)
2020-10-31 12:50:08: Start epoch 2:
2020-10-31 12:50:11: Epoch [2]   Batch [0]    Speed  18.4 (+1103) sample/sec  loss-ce = 1.04206  
2020-10-31 12:50:11: Epoch [2]   time cost: 3.39 sec (0.00 h)
2020-10-31 12:

KeyboardInterrupt: ignored